# Data Preperation

## Input:
- main_data_folder path from this file which will hold all data outside of git. This path folder is in the same folder that holds the GitHub folder of Diabetic-Retinopathy-Detection

##### Inside of main_data_folder there must be placed:
- 1) CSV of Targets for each image
- 2) train folder with all unzipped image data (38.1 GB total from all 5 train zipped files)

## Output:
- 1) downsized_data folder which includes 5 class folder that will be used for both training and future data augmentation. The images are 500 x 500 x 3 pixels each. The validate and test images are not in the folder.
- 2) data folder
- 3) Inside of the data folder there is a train, val, and test folder. The train folder is empty.
- 4) The val & test folder each have 71 randomely selected images for each class. These folders does not contain images in the downsized_data folder.

### Next Steps:
- Augment data in the downsized_data and put all new training data in the train folder

In [1]:
import numpy as np
import cv2
import os
import shutil
from distutils.dir_util import copy_tree
from pathlib import Path as path_check

# Input
- Main Folder of Data


In [2]:
main_data_folder = '../SML_Project_Data'

# Analysis of a Single Image

In [3]:
# Load an color image in color
img = cv2.imread( main_data_folder + '/train/10_left.jpeg',1)
print(img.shape)
img

(3168, 4752, 3)


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

### Create PNG of Image Object

In [4]:
cv2.imwrite('10_left_new.jpeg',img) # Confirmed manually the exported image is the same number of bytes as the original image

True

### Inspect Image Object

In [5]:
img.shape

(3168, 4752, 3)

In [6]:
img[1500][2300][1]

146

In [7]:
np.max(img)

255

In [8]:
np.min(img)

0

In [9]:
print('Original Size', img.shape)

print("%d bytes in original image" % (np.prod(img.shape)))

print('MB in the original array image is', img.size * img.itemsize / 1000000, 'versus the original image JPEG has 1.5 MB when the JPEG has comression versus a tensor') # 1 million bytes in a MB

resized = cv2.resize(img, (500, 500), interpolation = cv2.INTER_AREA)
print('\nResized Size', resized.shape)
cv2.imwrite('10_left_reshaped_new.jpeg',resized) # Goes from 1.5 MB to 3 KB which redices the size by a factor 1 K

print("%d bytes in compressed image" % (resized.size * resized.itemsize))

print('New pixel size is', np.prod(resized.shape))
print('Factor of reduction is by', np.prod(img.shape) / np.prod(resized.shape))

print('To get 100 K or so images in total, this would be around 5 times the original data set or a factor of original set of', np.prod(img.shape) / (np.prod(resized.shape) * 5 ) )
print('Results in GB of augmented data of', 50 / (np.prod(img.shape) / (np.prod(resized.shape) * 5 )))

Original Size (3168, 4752, 3)
45163008 bytes in original image
MB in the original array image is 45.163008 versus the original image JPEG has 1.5 MB when the JPEG has comression versus a tensor

Resized Size (500, 500, 3)
750000 bytes in compressed image
New pixel size is 750000
Factor of reduction is by 60.217344
To get 100 K or so images in total, this would be around 5 times the original data set or a factor of original set of 12.0434688
Results in GB of augmented data of 4.1516278100874064


# Downscale Images to 500 x 500 x 3 and Export

In [10]:
from pathlib import Path
import pandas as pd

In [11]:
train_target = pd.read_csv(main_data_folder + '/trainLabels.csv', delimiter=',')
print(len(train_target))
train_target.head()

35126


,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


### Count Images

In [12]:
total_images = 0
for index, row in train_target.iterrows():
    split_image = row['image'].split('_')
    image_num = split_image[0]
    side = split_image[1]
    
    image_name = str(image_num) + '_' + side + '.jpeg'

    train_image_path = main_data_folder + '/train/' + image_name

    my_file = Path(train_image_path)

    try:
        my_abs_path = my_file.resolve(strict=True)
    except FileNotFoundError:
        pass
    else:
        total_images = total_images + 1
total_images

35126

# Create Folders to Store All Downsized Data

In [14]:
def create_folder(path, replace = False):
    if replace:
        print('Attempting to delete folder of', path)
        try:
            shutil.rmtree(path)
        except FileNotFoundError:
            print('Folder already does not exist')
        else:
            pass
    
    print('Looking to create folders of', path)
    try:
        os.mkdir(path)
        print('')
    except FileExistsError:
        print('Folder Already Exists in ' + path + '\n')
    else:
        pass

# Create folders for each
downsized_data_path = main_data_folder + '/downsized_data'

create_folder(downsized_data_path)

Looking to create folders of ../SML_Project_Data/downsized_data
Folder Already Exists in ../SML_Project_Data/downsized_data



In [64]:
for disease in (0, 1, 2, 3, 4):
    path = downsized_data_path + '/resized_' + str(disease)
    create_folder(path)

Looking to create folders of ../SML_Project_Data/downsized_data/resized_0
Folder Already Exists in ../SML_Project_Data/downsized_data/resized_0

Looking to create folders of ../SML_Project_Data/downsized_data/resized_1
Folder Already Exists in ../SML_Project_Data/downsized_data/resized_1

Looking to create folders of ../SML_Project_Data/downsized_data/resized_2
Folder Already Exists in ../SML_Project_Data/downsized_data/resized_2

Looking to create folders of ../SML_Project_Data/downsized_data/resized_3
Folder Already Exists in ../SML_Project_Data/downsized_data/resized_3

Looking to create folders of ../SML_Project_Data/downsized_data/resized_4
Folder Already Exists in ../SML_Project_Data/downsized_data/resized_4



# Downsizing
All images in Train folder and moving images to different folders inside of downsized_data

In [15]:
# Distribute Images to each Folder
from datetime import datetime

counter = 0

status = int(total_images / 20)
print('Will give a status every', status, 'images')

new_shape = (500, 500)

for index, row in train_target.iterrows():
    
    start_time = datetime.now()
    
    split_image = row['image'].split('_')
    image_num = split_image[0]
    side = split_image[1]

    image_name = str(image_num) + '_' + side + '.jpeg'

    train_image_path = main_data_folder + '/train/' + image_name

    my_file = Path(train_image_path)

    try:
        my_file.resolve(strict=True)
    except FileNotFoundError:
        print('File not found')
        pass
    else:
        counter = counter + 1
        
        # Pull Image
        img2 = cv2.imread(train_image_path, 1)
        # Resize
        resized = cv2.resize(img2, new_shape, interpolation = cv2.INTER_AREA)
        # Create New Image File
        path = downsized_data_path + '/resized_' + str(row['level']) + '/' + image_name
        #print('Path:', path)
        cv2.imwrite(path, resized)
        resized
    if counter % status == 0:
        print('At', counter, 'images out of', total_images)
        
total_time = datetime.now() - start_time
print('To complete the script for this class took', total_time)
        
print('Processed Imnage Count', counter)

Will give a status every 1756 images
At 1756 images out of 35126
At 3512 images out of 35126
At 5268 images out of 35126
At 7024 images out of 35126
At 8780 images out of 35126
At 10536 images out of 35126
At 12292 images out of 35126
At 14048 images out of 35126
At 15804 images out of 35126
At 17560 images out of 35126
At 19316 images out of 35126
At 21072 images out of 35126
At 22828 images out of 35126
At 24584 images out of 35126
At 26340 images out of 35126
At 28096 images out of 35126
At 29852 images out of 35126
At 31608 images out of 35126
At 33364 images out of 35126
At 35120 images out of 35126
Processed Imnage Count 35126


# Counting Downsized Images

In [15]:
import os
import os, shutil

total = 0
min_file_count = np.inf
for disease_type in (0, 1, 2, 3, 4):
    path, dirs, files = next(os.walk(downsized_data_path + '/resized_' + str(disease_type)))
    file_count = len(files)
    if file_count < min_file_count:
        min_file_count = file_count
    print('Disease Class of', disease_type, 'has count of images of', file_count)
    total = total + file_count
print('Mininum count by class is', min_file_count)
    
train_file_count = int(min_file_count * .80)
print('Training set will have', train_file_count)
val_file_count = min_file_count - train_file_count
print('Validation & Test set will each have an image count of', int(val_file_count/2))

print('In Total there are', total, 'images in downsized_data folder')

Disease Class of 0 has count of images of 25668
Disease Class of 1 has count of images of 2301
Disease Class of 2 has count of images of 5150
Disease Class of 3 has count of images of 731
Disease Class of 4 has count of images of 566
Mininum count by class is 566
Training set will have 452
Validation & Test set will each have an image count of 57
In Total there are 34416 images in downsized_data folder


## Creating Folders for Training, Validation and Test Set

In [66]:
main_data_folder

'../SML_Project_Data'

In [67]:
create_folder(main_data_folder + '/data', replace = True)
create_folder(main_data_folder + '/data/train', replace = True)
create_folder(main_data_folder + '/data/val', replace = True)
create_folder(main_data_folder + '/data/test', replace = True)

# Create Training Set
for disease in (0, 1, 2, 3, 4):
    for type_path in ('train','val','test'):
        path = main_data_folder + '/data/' + type_path + '/resized_' + str(disease)
        create_folder(path, replace = True)

Attempting to delete folder of ../SML_Project_Data/data
Looking to create folders of ../SML_Project_Data/data

Attempting to delete folder of ../SML_Project_Data/data/train
Folder already does not exist
Looking to create folders of ../SML_Project_Data/data/train

Attempting to delete folder of ../SML_Project_Data/data/val
Folder already does not exist
Looking to create folders of ../SML_Project_Data/data/val

Attempting to delete folder of ../SML_Project_Data/data/test
Folder already does not exist
Looking to create folders of ../SML_Project_Data/data/test

Attempting to delete folder of ../SML_Project_Data/data/train/resized_0
Folder already does not exist
Looking to create folders of ../SML_Project_Data/data/train/resized_0

Attempting to delete folder of ../SML_Project_Data/data/val/resized_0
Folder already does not exist
Looking to create folders of ../SML_Project_Data/data/val/resized_0

Attempting to delete folder of ../SML_Project_Data/data/test/resized_0
Folder already does not

# Copy all files from downsized_data_path to a saved folder

In [68]:
save_downsized_images = False

if save_downsized_images:
    print('Duplicating:' ,downsized_data_path)
    path = downsized_data_path + '_saved'
    print(path)

    copy_tree(src=downsized_data_path, dst=path, verbose=0)

# Move 10% each of Min Class Data from downsized_data_path to val and test (20% total)
## Randomize train_target object, but keep sample individual next to each other

In [72]:
import random

train_target = pd.read_csv(main_data_folder + '/trainLabels.csv', delimiter=',')

train_target[['id','side']] = train_target['image'].str.split(pat = "_", expand=True)

groups = [train_target for _, train_target in train_target.groupby('id')]

random.Random(14).shuffle(groups)

train_target = pd.concat(groups).reset_index(drop=True)
train_target.head()

,image,level,id,side
0,24779_left,2,24779,left
1,24779_right,2,24779,right
2,26490_left,0,26490,left
3,26490_right,0,26490,right
4,40232_left,2,40232,left


In [73]:
train_target.drop(['id', 'side'], axis=1, inplace = True)
train_target.head()

,image,level
0,24779_left,2
1,24779_right,2
2,26490_left,0
3,26490_right,0
4,40232_left,2


## Move first records in each class to the validate folder

In [74]:
def move_files_val_test(downsized_data_path,
                        main_data_folder,
                        subfolder,
                        val_file_count,
                        move = False,
                       verbose = False):

    counter = 0

    track ={
        0 : 0,
        1 : 0,
        2 : 0,
        3 : 0,
        4 : 0
        }
    missing = 0

    if move == False:
        print('Copying and not moving files')
    
    for index, row in train_target.iterrows():

        # Get training set path of next image
        split_image = row['image'].split('_')
        image_num = split_image[0]
        side = split_image[1]
        image_name = str(image_num) + '_' + side + '.jpeg'
        downsized_image_path = downsized_data_path + '/resized_' + str(row['level']) + '/' + image_name
        
        # Check that the class of interest isn't full
        if track.get(row['level']) < int(val_file_count/2):
            if verbose:
                print('required images is not full')
        
            # Check if file exists
            if path_check(downsized_image_path).is_file():

                # Create new image file path
                output_path = main_data_folder + '/data/' + subfolder + '/resized_' + str(row['level']) + '/' + image_name

                if move:
                    shutil.move(src=downsized_image_path, dst=output_path)
                else:
                    print('Moving image from', downsized_image_path, 'to', output_path)
                    shutil.copyfile(src=downsized_image_path, dst=output_path)
                counter = counter + 1

                # update counter on folder
                new_value = track.get(row['level']) + 1
                track.update({row['level'] : new_value})
            else:
                missing = missing + 1

    print('Processed Image Count', counter)

    print(track, 'Missing images are', missing )

for subfolder in ('val','test'):
    move_files_val_test(downsized_data_path=downsized_data_path,
                            main_data_folder=main_data_folder,
                            subfolder=subfolder,
                            val_file_count=val_file_count,
                            move = True,
                           verbose = False)

Processed Image Count 355
{0: 71, 1: 71, 2: 71, 3: 71, 4: 71} Missing images are 0
Processed Image Count 355
{0: 71, 1: 71, 2: 71, 3: 71, 4: 71} Missing images are 355


# If error, moves saved downsized images back to the downsized images folder.
This will result in all images being in the folder again. This does not fun if error = False

In [75]:
error = False

if error:
    path = downsized_data_path + '_saved'
    copy_tree(src=downsized_data_path + '_saved', dst=downsized_data_path, verbose=0)

## Utilized to get comparison of no data augmentation or resampling with full data augmentation

In [ ]:
def move_x_files_train(downsized_data_path,
                        main_data_folder,
                        subfolder,
                        train_file_count=566,
                        move = False,
                       verbose = False):

    counter = 0

    track ={
        0 : 0,
        1 : 0,
        2 : 0,
        3 : 0,
        4 : 0
        }
    missing = 0

    if move == False:
        print('Copying and not moving files')
    
    for index, row in train_target.iterrows():

        # Get training set path of next image
        split_image = row['image'].split('_')
        image_num = split_image[0]
        side = split_image[1]
        image_name = str(image_num) + '_' + side + '.jpeg'
        downsized_image_path = downsized_data_path + '/resized_' + str(row['level']) + '/' + image_name
        if verbose:
            print(downsized_image_path)
        
        # Check that the class of interest isn't full
        if track.get(row['level']) < train_file_count:
            if verbose:
                print('required images is not full')
        
            # Check if file exists
            if path_check(downsized_image_path).is_file():

                # Create new image file path
                output_path = main_data_folder + '/data/train/resized_' + str(row['level']) + '/' + image_name

                if move:
                    shutil.move(src=downsized_image_path, dst=output_path)
                else:
                    if verbose:
                        print('Moving image from', downsized_image_path, 'to', output_path)
                    shutil.copyfile(src=downsized_image_path, dst=output_path)
                counter = counter + 1

                # update counter on folder
                new_value = track.get(row['level']) + 1
                track.update({row['level'] : new_value})
            else:
                missing = missing + 1

    print('Processed Image Count', counter)

    print(track, 'Missing images are', missing )

move_x_files_train(downsized_data_path=downsized_data_path,
                        main_data_folder=main_data_folder,
                        subfolder=subfolder,
                        train_file_count=566,
                        move = False,
                       verbose = False)